# MNIST For ML Beginners

## MNIST数据集

下面两行代码会自动下载、读取MNIST数据集。

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


MNIST数据集分三部分：训练集(mnist.train)有55000个数据点，测试集(mnist.test)有10000个数据点，验证集(mnist.validatioin)有5000个数据点。

每个数据点包含两部分：一张图片和对应的label值。我们称图片为"x",labels为"y"。

比如，训练集图像是`mnist.train.images`，训练集labels是`mnist.train.labels`。

图片大小是28\*28，可以把它看成一个矩阵。进一步，展开成28\*28=784的向量。

`mnist.train.images`是一个shape为[55000, 784]的tensor。

每一个图片都有一个label，值是0-9。

我们把label值用“one-hot vector”形式来表示。什么是one-hot vector？首先他是一个向量，只有一个元素值是1，其余都是0.

比如，3转化成[0,0,0,1,0,0,0,0,0,0]。

因此，`mnist.train.labels`的shape是[55000, 10]。

有了数据集，就可以创建模型了。

## Softmax Regressions

我们知道MNIST中每张图片的label取值范围是[0, 9]。所以预测图片的label值是一个典型的分类问题。

最简单的多分类模型是Softmax Regression。它为每个类型分配一个概率值，所有类别的概率值和是1。**以后我们会创建许多更复杂的模型，但通常，最后一层都是softmax**。

softmax函数本质上是一个归一化的指数函数。

## 模型实现

我们通常使用Numpy库处理数值计算比如矩阵操作。但是，如果用GPU进行计算，然后把数据传到CPU，这时，不同设备之间的传值操作便成为了影响计算性能的瓶颈。

Tensorflow是如何解决这个问题呢？整个计算图都独立于Python。

In [2]:
import tensorflow as tf

In [3]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

W的shape是[784, 10]，因为W要和长度784的图片向量相乘，并且产生10维向量值。

In [4]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

## 训练

损失函数我们使用交叉熵(cross-entropy)。

In [5]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [6]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [9]:
sess = tf.InteractiveSession()

In [10]:
tf.global_variables_initializer().run()

In [12]:
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})

## 评估模型

如何评估模型的好坏？

In [13]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))

得到boolean列表。

In [14]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9202
